In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
from itertools import compress
import shutil

In [6]:
#declarations of the paths and constants of the simulations
sim_name = 'sample_simulation'
file_copy_list2 = ['j1.png', 'j2.png', 'j3.png' , 'j4.png', 'j5.png' , 'not.png']# , 'm000100.ovf']
file_copy_list1= ['allmakemovie2j.m']
application_path = 'D:\\viewer\\application\\'
file_copy_list2 = list()
localpath = 'D:\\viewer\\simulations\\'+ sim_name + '\\'
remotepath = '/scratch/users/acheghabouri17/simulations/' + sim_name + '/' 
imagepath = 'paper_final\\'
processing= 'tclsh /kuacc/apps/oommf/1.2a5/oommf.tcl avf2ppm -format b24 -config "/scratch/users/acheghabouri17/workfolder/generator/processing.config" *.ovf'
normal= 'tclsh /kuacc/apps/oommf/1.2a5/oommf.tcl avf2ppm -f -format b24 -config "/scratch/users/acheghabouri17/workfolder/generator/setting1.config" *.ovf'
pref = 'cur'

# Pt- Cobalt

In [3]:
Msat = np.arange(400000, 700000, 20000)
# msat=np.array([580000])
D = np.arange(1,3.5, .2)
D = D*1e-3
Aex = np.arange(1.4, 2.5 , 0.2)
Aex = Aex*1e-12
JJ_stage1 = -np.arange(0 , 2, 0.05)
JJ_stage1 = JJ_stage1*1e12
JJ_push = -np.linspace(0.1 , 1.2 , 10)
JJ_push = JJ_push*1e12
Bz = np.arange(-0.1,0.1, 0.02)
JJ_clear = -np.arange(0.1, 5 , 0.1)*1e12
Nz = [4 , 6]
Ku1 = np.linspace(0.5e6,1.2e6,10);
alpha = np.linspace(0.01 , 1 , 10)
sweep_parameters = {'Msat':Msat, 'JJ':JJ_push}
other_parameters = { 'Ku1':'8e5',
                    'xi':'0.3',
                    'alpha':'0.3',
                    'Dind':'3.5e-3',
                    'Bx':'0', 'By':'0', 'Bz':'0', 
                    'anisU':'vector(0, 0, 1)',
                    'Aex':'15e-12' ,'timestep':'1e-11'}
sweep_keys = list(sweep_parameters.keys())

# Total Number Of simulations

In [4]:
from functools import reduce
from operator import mul
reduce(mul , [len(a) for a in sweep_parameters.values()])

150

In [22]:
a = [x.endswith('.mx3') for x in os.listdir(localpath)]
mumax_files = list(compress(os.listdir(localpath) , a))
a = [x.endswith('.sh') for x in os.listdir(localpath)]
bash_files = list(compress(os.listdir(localpath) , a))
print(mumax_files)
print(bash_files)

['complete_1_0.mx3']
['complete_1_0.sh']


In [23]:
mumax_strings = list()
for mumax_file in mumax_files: 
    f = open(localpath + mumax_file , 'r')
    file = f.read()
    for parameter in other_parameters: 
        file = file.replace('the_'+parameter ,other_parameters[parameter] )
    mumax_strings.append(file)
    f.close()

In [24]:
f = open(localpath + bash_files[0])
bash_string = f.read()
f.close()

In [25]:
sweep_names = list(sweep_parameters.keys())
tile_parameters = {'stage_name':"'complete_1_0'",
              'outer':"'" + sweep_names[0] + "'",
              'inner':"'" + sweep_names[1] + "'",
               'frame_range':'[1,100]', 
               'tile_size':'[100,100]',
                  'folder_name':"'" + sim_name + "'"}
with open(localpath+'parameters.txt' , 'w') as parameters_file:
    tile_statement = [x+'='+y for x,y in zip(tile_parameters.keys(), tile_parameters.values())]
    tile_statement = ';'.join(tile_statement) + ';'
    parameters_file.write(tile_statement)

In [26]:
directives = ['']*len(mumax_files)
directives_file = open(localpath + 'directives.txt' , 'w')
pre_string = sweep_keys[0]
pre_string2 = sweep_keys[1]
for item in sweep_parameters[sweep_keys[0]]:
    item_string = "%1.4e"% item
    path = localpath + pre_string + '=' + item_string
    first_folder = pre_string + '=' + item_string
    os.mkdir( path )
    for second_item in sweep_parameters[sweep_keys[1]]:
        mlocalpath = localpath + pre_string + '=' + item_string + '\\'
        folder_name = pre_string2 +'=' + "%1.4e" % second_item
        os.chdir(mlocalpath)
        os.mkdir(folder_name)
        print(mlocalpath)
        print(folder_name)
        for cfile in file_copy_list1:
            print('copy '+application_path  + cfile+ ' '+ mlocalpath+folder_name + '\\'+cfile)
            
            #subprocess.call('copy '+application_path  + cfile+ ' '+ mlocalpath+folder_name + '\\'+cfile, shell=True)
            shutil.copyfile(application_path  + cfile , mlocalpath+folder_name + '\\'+cfile )
        for cfile in file_copy_list2:
            #subprocess.call('copy '+application_path + imagepath + cfile+ ' '+ mlocalpath+folder_name+ '\\'+cfile, shell=True)
            shutil.copyfile(application_path + imagepath + cfile, mlocalpath+folder_name+ '\\'+cfile)
            print('copy '+application_path + imagepath + cfile+ ' '+ mlocalpath+folder_name+ '\\'+cfile)
        for enumerator , mumax_file in enumerate(mumax_files):
            #print(mlocalpath)
            print(mumax_file)
            directives[enumerator] = directives[enumerator]+ 'cd ' + remotepath +first_folder + '/'+ folder_name + '/'
            directives[enumerator] += '\n sbatch '+ mumax_file[:-4] + '.sh\n ' 
            mumax_string = mumax_strings[enumerator]
            mumax_file_temp = mumax_string.replace('the_' + pre_string ,"%1.4e" % item )
            mumax_file_temp = mumax_file_temp.replace('the_'+pre_string2 , "%1.4e" % second_item)
            #print(file+ '.mx3');
            f = open(folder_name + '\\' + mumax_file , 'w')
            f.write(mumax_file_temp)
            f.close()
            bash_file_temp = bash_string.replace('the_file_name', mumax_file[:-4])
            f = open(folder_name + '\\' +mumax_file[:-4] + '.sh' , 'wb')
            f.write(bash_file_temp.encode())
            f.close()
            
            #print(second_item)
            #print(enumerator)
directives_file.write(''.join(directives))
directives_file.close()
os.chdir('..')

D:\viewer\simulations\sample_simulation\Msat=4.0000e+05\
JJ=-1.0000e+11
copy D:\viewer\application\allmakemovie2j.m D:\viewer\simulations\sample_simulation\Msat=4.0000e+05\JJ=-1.0000e+11\allmakemovie2j.m
complete_1_0.mx3
D:\viewer\simulations\sample_simulation\Msat=4.0000e+05\
JJ=-2.2222e+11
copy D:\viewer\application\allmakemovie2j.m D:\viewer\simulations\sample_simulation\Msat=4.0000e+05\JJ=-2.2222e+11\allmakemovie2j.m
complete_1_0.mx3
D:\viewer\simulations\sample_simulation\Msat=4.0000e+05\
JJ=-3.4444e+11
copy D:\viewer\application\allmakemovie2j.m D:\viewer\simulations\sample_simulation\Msat=4.0000e+05\JJ=-3.4444e+11\allmakemovie2j.m
complete_1_0.mx3
D:\viewer\simulations\sample_simulation\Msat=4.0000e+05\
JJ=-4.6667e+11
copy D:\viewer\application\allmakemovie2j.m D:\viewer\simulations\sample_simulation\Msat=4.0000e+05\JJ=-4.6667e+11\allmakemovie2j.m
complete_1_0.mx3
D:\viewer\simulations\sample_simulation\Msat=4.0000e+05\
JJ=-5.8889e+11
copy D:\viewer\application\allmakemovie2j.m 

D:\viewer\simulations\sample_simulation\Msat=5.2000e+05\
JJ=-2.2222e+11
copy D:\viewer\application\allmakemovie2j.m D:\viewer\simulations\sample_simulation\Msat=5.2000e+05\JJ=-2.2222e+11\allmakemovie2j.m
complete_1_0.mx3
D:\viewer\simulations\sample_simulation\Msat=5.2000e+05\
JJ=-3.4444e+11
copy D:\viewer\application\allmakemovie2j.m D:\viewer\simulations\sample_simulation\Msat=5.2000e+05\JJ=-3.4444e+11\allmakemovie2j.m
complete_1_0.mx3
D:\viewer\simulations\sample_simulation\Msat=5.2000e+05\
JJ=-4.6667e+11
copy D:\viewer\application\allmakemovie2j.m D:\viewer\simulations\sample_simulation\Msat=5.2000e+05\JJ=-4.6667e+11\allmakemovie2j.m
complete_1_0.mx3
D:\viewer\simulations\sample_simulation\Msat=5.2000e+05\
JJ=-5.8889e+11
copy D:\viewer\application\allmakemovie2j.m D:\viewer\simulations\sample_simulation\Msat=5.2000e+05\JJ=-5.8889e+11\allmakemovie2j.m
complete_1_0.mx3
D:\viewer\simulations\sample_simulation\Msat=5.2000e+05\
JJ=-7.1111e+11
copy D:\viewer\application\allmakemovie2j.m 

D:\viewer\simulations\sample_simulation\Msat=6.6000e+05\
JJ=-2.2222e+11
copy D:\viewer\application\allmakemovie2j.m D:\viewer\simulations\sample_simulation\Msat=6.6000e+05\JJ=-2.2222e+11\allmakemovie2j.m
complete_1_0.mx3
D:\viewer\simulations\sample_simulation\Msat=6.6000e+05\
JJ=-3.4444e+11
copy D:\viewer\application\allmakemovie2j.m D:\viewer\simulations\sample_simulation\Msat=6.6000e+05\JJ=-3.4444e+11\allmakemovie2j.m
complete_1_0.mx3
D:\viewer\simulations\sample_simulation\Msat=6.6000e+05\
JJ=-4.6667e+11
copy D:\viewer\application\allmakemovie2j.m D:\viewer\simulations\sample_simulation\Msat=6.6000e+05\JJ=-4.6667e+11\allmakemovie2j.m
complete_1_0.mx3
D:\viewer\simulations\sample_simulation\Msat=6.6000e+05\
JJ=-5.8889e+11
copy D:\viewer\application\allmakemovie2j.m D:\viewer\simulations\sample_simulation\Msat=6.6000e+05\JJ=-5.8889e+11\allmakemovie2j.m
complete_1_0.mx3
D:\viewer\simulations\sample_simulation\Msat=6.6000e+05\
JJ=-7.1111e+11
copy D:\viewer\application\allmakemovie2j.m 

In [14]:
os.chdir('..')